## Initialize the libraries and origin and destination workspaces

In [ ]:
#This is only one time run
!pip install azure-cognitiveservices-vision-customvision

In [2]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

# DESTINATION Resource Group Keys
dest_training_key = "<YOUR TRAINING KEY>"
dest_prediction_key = "<YOUR PREDICTION KEY>"
dest_project_new_name = "DebrisRecognition_Copy"

# ORIGIN Resource Group Keys
training_key = "70b0304d1e69481693e7d5e249212adf"
prediction_key = "7e1dcf4b76114b96a4e00e728a8cc6d7"

trainer = training_api.TrainingApi(training_key)
dest_trainer = training_api.TrainingApi(dest_training_key)

# Find the image classification domain
classification_domain = next(domain for domain in trainer.get_domains() if domain.type == "Classification")
dest_classification_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "Classification")

## Get the origin project ID reference

In [3]:
myProjects = trainer.get_projects()

In [4]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

DebrisRecognition
aadaa47b-ca7b-4536-8450-ec4428dd302e



In [5]:
Project = trainer.get_project(project_id="aadaa47b-ca7b-4536-8450-ec4428dd302e")

## Create the destination Project 

In [6]:
dest_Project = dest_trainer.create_project(dest_project_new_name, domain_id=dest_classification_domain.id)

In [7]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

DebrisRecognition_Copy
13ac2269-770d-4fd2-8242-c90a21dfbf4c

DebrisRecognition
aadaa47b-ca7b-4536-8450-ec4428dd302e



## Get the tags on origin project and create same tags on destination project

In [8]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

Ocean
Object


In [9]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

{'Ocean': '49043d8b-f6d6-499e-84fb-af4bc487005b', 'Object': '0d098190-75a2-46e6-a7b1-29070ef4bdd9'}
['49043d8b-f6d6-499e-84fb-af4bc487005b', '0d098190-75a2-46e6-a7b1-29070ef4bdd9']


## Get the images on origin project

In [10]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [11]:
tagged_images_with_tags = []
for image in tagged_images: #for each tagged image on origin
    dest_tags_ids = []
    
    for tag in image.tags: #for each tag on the origin image
        dest_tags_ids.append(dest_tags_dict[tag.tag_name]) #append it to the image dest_tags_ids list
    
    tagged_images_with_tags.append(ImageUrlCreateEntry(url=image.image_uri, tag_ids=dest_tags_ids))
print("Done")

Done


## Create the images with regions on destination project

In [12]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_tags)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_tags[begin:end])

In [13]:
print("Count of Tagged images on origin project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

Count of Tagged images on origin project: 103
Count of Tagged images on destination project: 103


## Train the model on destination Project

In [14]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

Training... (showing status each 10 seconds)
Training status: Training
Training status: Training
Training status: Completed
Done!


## Get performance metrics

In [15]:
performance = dest_trainer.get_iteration_performance(dest_Project.id, iteration.id, threshold=0.80)
print("Iteration: " + iteration.name)
print("Precision: " + str(performance.precision * 100)+ "%")
print("Recall: " + str(performance.recall * 100) + "%")

Iteration: Iteration 1
Precision: 92.01973679999999%
Recall: 88.34734%


In [16]:
per_tag_performance = performance.per_tag_performance

In [17]:
for tag in per_tag_performance:
    print ("Tag: " + tag.name + "\t\t" + "Precision: " + str(tag.precision) + "\t" + "Recall: " + str(tag.recall))

Tag: Object		Precision: 0.969697	Recall: 0.841269851
Tag: Ocean		Precision: 0.881481469	Recall: 0.948717952


## Get and use the default prediction endpoint

In [2]:
#You can make a web search online for "ocean aerial dolphin" and get an URL:
url = "https://s3.envato.com/files/239350427/preview.jpg"

<div style="width:100%; margin-left:auto; margin-right:auto; margin-bottom:5px; margin-top:17px;">
<img src="https://s3.envato.com/files/239350427/preview.jpg" alt="IMAGE" /><br>
</div>

In [19]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = prediction_endpoint.PredictionEndpoint(dest_prediction_key)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [20]:
# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.3): # only show the prediction with more than 30% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Object: 92.83%
	Ocean: 36.21%
